v3

In [13]:
from bs4 import BeautifulSoup
import requests
import time
from IPython.display import clear_output
from collections import namedtuple

# Define a namedtuple for storing course row data
CourseRow = namedtuple('CourseRow', [
    'code',
    #'program',
    'faculty',
    #'credit',
    'section',
    'day_time_room',
    #'total_seat',
    #'booked_seat',
    'remaining_seat'
])

def read_courses():
    # Read courses from a file into a dictionary with sets for sections
    with open("course_input.txt", 'r') as f1:
      courses = {}
      for line in f1.readlines():
        parts = line.strip().split()
        course_code = parts[0]

        if len(parts) > 1:
            # Use set for faster lookups
            sections = set(parts[1:])
        else:
            sections = set()  # No sections provided, check all sections (empty set)

        courses[course_code] = sections  # Store sections as a set

    return courses

def get_table_data(url):
  try:
    # Scraping the website HTML
    page = requests.get(url)
    page.raise_for_status()  # Handle request errors
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find("table", attrs={"id": "customers"})
    if not table:
      print("Table not found on page.")
      return []

    # Extract rows from the table
    return table.find_all("tr")

  except requests.RequestException as e:  #Error handling
    print(f"Error fetching the URL: {e}")
    return []

  # Headers:
    # *nbsp*
    # Course code
    # Program
    # Faculty
    # Credit
    # Section
    # Day, Time, Room
    # Total Seat
    # Seat Booked
    # Seat Remaining

def check_seat_availability(courses, table_data):
  available = []

  for row in table_data:
    td = row.find_all("td")

    # Skip header rows or incomplete rows
    if len(td) < 10:
      continue

    # Use namedtuple to store the row data
    course = CourseRow(
        code=td[1].text.strip(),
        # program=td[2].text.strip(),
        faculty=td[3].text.strip(),
        # credit=td[4].text.strip(),
        section=td[5].text.strip(),
        day_time_room=td[6].text.strip(),
        # total_seat=td[7].text.strip(),
        # booked_seat=td[8].text.strip(),
        remaining_seat=td[9].text.strip()
    )

    # Check if the course is in the target list
    if course.code in courses:
      target_sections = courses[course.code]

      # Check specific sections or all sections (if section set is empty)
      if not target_sections or course.section in target_sections:

        # If section list is empty or section matches, check seat availability
        if int(course.remaining_seat) > 0:
          available.append(course)

  return available

def display_results(available, headers):
  clear_output(wait=False)
  if available:
    print('Available Courses:')
    for course in available:
      print("#===================================#")
      print("Current Time:", time.strftime("%H:%M:%S", time.localtime(time.time() + 6*3600)))
      print(headers[0] + ':', course.code)
      print(headers[1] + ':', course.section)
      print(headers[3] + ':', course.day_time_room)
      print(headers[2] + ':', course.remaining_seat)
      print("#===================================#\n")
  else:
    print("No seats currently available...")

def periodic_check(interval, url, courses):
  headers = ['Course', 'Section', 'Available Seats', 'Day, Time, Room']
  while True:
    table_data = get_table_data(url)
    available = check_seat_availability(courses, table_data)
    display_results(available, headers)
    time.sleep(interval)

def main():
  url = "https://usis.bracu.ac.bd/academia/admissionRequirement/getAvailableSeatStatus"
  courses = read_courses()
  interval = 90 #seconds
  periodic_check(interval, url, courses)

main()

Available Courses:
#===================================#
Current Time: 12:58:47
Course: ACT201
Section: 12
Day, Time, Room: Su(05:00 PM-06:20 PM-07A-04C) Tu(05:00 PM-06:20 PM-07A-04C)
Available Seats: 1
#===================================#

#===================================#
Current Time: 12:58:47
Course: ACT201
Section: 14
Day, Time, Room: Su(08:00 AM-09:20 AM-UB0000) Tu(08:00 AM-09:20 AM-UB0000)
Available Seats: 3
#===================================#

#===================================#
Current Time: 12:58:47
Course: ACT201
Section: 15
Day, Time, Room: Su(09:30 AM-10:50 AM-UB0000) Tu(09:30 AM-10:50 AM-UB0000)
Available Seats: 2
#===================================#

#===================================#
Current Time: 12:58:47
Course: ACT201
Section: 16
Day, Time, Room: Sa(11:00 AM-12:20 PM-UB0000) Th(11:00 AM-12:20 PM-UB0000)
Available Seats: 3
#===================================#

#===================================#
Current Time: 12:58:47
Course: ACT201
Section: 17
Day, Ti

KeyboardInterrupt: 